# Configuración (importar dependencias, librerías, ...)

In [6]:
# Set the seed value all over the place to make this reproducible.
# esto hay que ponerlo justo antes de importar para que los experimentos
# sean reproducible
!pip3 install emoji==0.6.0
!pip install sentencepiece
!pip install pytorch-lightning
!pip install google-colab
import random
import torch
import numpy as np
import os
import re
from pytorch_lightning import seed_everything
from google.colab import drive

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)# Store the average loss after eachepoch so we can plot them.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["TF_DETERMINISTIC_OPS"] = "1" # See:https://github.com/NVIDIA/tensorflow-determinism#confirmed-current-gpu-specific-sources-of-non-determinism-with-solutions
seed_everything(42, workers=True)

!pip install transformers datasets
#!pip install textblob
#from google.colab import drive
from datasets import Dataset, DatasetDict, load_metric
import pandas as pd
import sklearn as sk
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
 TrainingArguments, Trainer, pipeline, EarlyStoppingCallback

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


INFO:lightning_fabric.utilities.seed:Global seed set to 42


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# Check that pyTorch is identifying the GPU
if torch.cuda.device_count() > 0:
  print(f'GPU detected. Currently using: "{torch.cuda.get_device_name(0)}"')
else:
  print('Currently using CPU, change the type of the runtime in the \'runtime\' tab')

GPU detected. Currently using: "Tesla T4"


# Preparación de los datos

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


## Lectura de los ficheros

In [9]:
# Cargamos los datos de entrenamiento
train_data_path = '/content/drive/MyDrive/EDOS/train_all_tasks.csv'
# Lo pasamos a un dataframe
train_df = pd.read_csv(train_data_path, encoding = 'UTF-8', sep=',')
#entrenar con train test y validacion con split
train_df_full  = train_df.rename(columns={"label_sexist":"label"}) # poner el campo a utilizar
print("distribucion original: ",train_df_full.value_counts("label"))
df_0 = train_df_full[train_df_full["label"]=='not sexist'][0:300]
df_1 = train_df_full[train_df_full["label"]=='sexist'][0:300]

train_df_full = pd.concat([df_0,df_1])
print(train_df_full)
print("distribucion despues del undersampling: ",train_df_full.value_counts("label"))


train_df, auxiliar_df = train_test_split(train_df_full, test_size = 0.20, shuffle = True, stratify=train_df_full[['label']])
valid_df, test_df = train_test_split(auxiliar_df, test_size = 0.20, shuffle = True, stratify=auxiliar_df[['label']])

print("Ejemplos del conjunto completo: ", len(train_df_full))
print("Ejemplos usados para entrenar: ", len(train_df))
print("Ejemplos usados para validar: ", len(valid_df))
print("Ejemplos usados para test: ", len(test_df))
train_df_full.value_counts('label')
train_df.value_counts('label')
valid_df.value_counts('label')
test_df.value_counts('label')


distribucion original:  label
not sexist    10602
sexist         3398
dtype: int64
                     rewire_id  \
0      sexism2022_english-7358   
1      sexism2022_english-2367   
2      sexism2022_english-3073   
3     sexism2022_english-14895   
4      sexism2022_english-4118   
...                        ...   
1241   sexism2022_english-1255   
1243  sexism2022_english-14104   
1244   sexism2022_english-7205   
1245   sexism2022_english-9883   
1247   sexism2022_english-6485   

                                                   text       label  \
0                 Damn, this writing was pretty chaotic  not sexist   
1     Yeah, and apparently a bunch of misogynistic v...  not sexist   
2          How the FUCK is this woman still an MP!!!???  not sexist   
3     Understand. Know you're right. At same time I ...  not sexist   
4        Surprized they didn't stop and rape some women  not sexist   
...                                                 ...         ...   
1241  If yo

label
not sexist    12
sexist        12
dtype: int64

## Limpieza de datos

In [10]:
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.replace('[url]', '')
    tweet = tweet.replace('[link]', '')   # remove [link]
    tweet = re.sub(r'pic.twitter\S+', '', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    tweet = tweet.replace('[user]', '')   # remove [link]
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

In [11]:
train_df
#aqui hay que añadir las demas funciones de limpieza y las de tratamiento de los emojis
train_df['text'] = train_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()
valid_df['text'] = valid_df['text'].str.lower()

#Funciones de limpieza
#train_df['text'] = train_df['text'].apply(limpia_tweet)
#valid_df['text'] = valid_df['text'].apply(limpia_tweet)
#test_df['text'] = test_df['text'].apply(limpia_tweet)

train_df['text'] = train_df['text'].apply(remove_links)
train_df['text'] = train_df['text'].apply(remove_users)
train_df['text'] = train_df['text'].apply(remove_hashtags)
train_df['text'] = train_df['text'].apply(remove_av)

valid_df['text'] = valid_df['text'].apply(remove_links)
valid_df['text'] = valid_df['text'].apply(remove_users)
valid_df['text'] = valid_df['text'].apply(remove_hashtags)
valid_df['text'] = valid_df['text'].apply(remove_av)

test_df['text'] = test_df['text'].apply(remove_links)
test_df['text'] = test_df['text'].apply(remove_users)
test_df['text'] = test_df['text'].apply(remove_hashtags)
test_df['text'] = test_df['text'].apply(remove_av)

train_df

,rewire_id,text,label,label_category,label_vector
128,sexism2022_english-1375,"untouched not after 6 years, especially not if...",not sexist,none,none
1009,sexism2022_english-12805,take the worst guy playing in the nba and plop...,sexist,3. animosity,3.2 immutable gender differences and gender st...
296,sexism2022_english-3914,we have now just become the that has been liv...,not sexist,none,none
838,sexism2022_english-5291,"and there we have the root cause of the ""white...",sexist,3. animosity,3.2 immutable gender differences and gender st...
191,sexism2022_english-13799,unbelievable!!!!! university of washington: pr...,not sexist,none,none
...,...,...,...,...,...
244,sexism2022_english-4257,regardless if you were a horny little bastard ...,not sexist,none,none
897,sexism2022_english-8097,and she will never put 2 and 2 together to rea...,sexist,3. animosity,"3.1 casual use of gendered slurs, profanities,..."
148,sexism2022_english-9768,this girl's pretty fucking smart and on the mo...,not sexist,none,none
672,sexism2022_english-8240,at least the guy was right for calling her a f...,sexist,3. animosity,3.2 immutable gender differences and gender st...


In [12]:
# Convertimos los dataframes en objetos datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

#print(train_dataset)
#print(train_dataset, valid_dataset)


In [13]:
# Mostramos los datos en formato pandas
train_dataset.set_format("pandas")
train_dataset[:]

,rewire_id,text,label,label_category,label_vector,__index_level_0__
0,sexism2022_english-1375,"untouched not after 6 years, especially not if...",not sexist,none,none,128
1,sexism2022_english-12805,take the worst guy playing in the nba and plop...,sexist,3. animosity,3.2 immutable gender differences and gender st...,1009
2,sexism2022_english-3914,we have now just become the that has been liv...,not sexist,none,none,296
3,sexism2022_english-5291,"and there we have the root cause of the ""white...",sexist,3. animosity,3.2 immutable gender differences and gender st...,838
4,sexism2022_english-13799,unbelievable!!!!! university of washington: pr...,not sexist,none,none,191
...,...,...,...,...,...,...
475,sexism2022_english-4257,regardless if you were a horny little bastard ...,not sexist,none,none,244
476,sexism2022_english-8097,and she will never put 2 and 2 together to rea...,sexist,3. animosity,"3.1 casual use of gendered slurs, profanities,...",897
477,sexism2022_english-9768,this girl's pretty fucking smart and on the mo...,not sexist,none,none,148
478,sexism2022_english-8240,at least the guy was right for calling her a f...,sexist,3. animosity,3.2 immutable gender differences and gender st...,672


In [14]:
# Reseteamos el formato para que no haya fallos
train_dataset.reset_format() 
valid_dataset.reset_format()

# Borramos los dataframes puesto que no los vamos a usar más
#del train_df_palabras
#del train_df_full
#del train_df
#del test_df

# Preparación de los conjuntos para el entrenamiento

In [15]:
# Asignamos una etiqueta numérica en función de la etiqueta principal
#   Label = 0 --> negativo
#   Label = 1 --> positivo
# Para EDOS es 'label_sexist'
def set_labels(records):
  if records['label'] == 'not sexist':
    label = 0
  else:
    label = 1
  return {'labels': label}

In [16]:
# Correct the labels of the test split and create a Dataset dict for all the splits (except test_dataset)
# Esto es para cuando tengamos fichero de train y de valid
# dataset = DatasetDict({'train': train_dataset, 'valid': valid_dataset})  
dataset_train = train_dataset
dataset_valid = valid_dataset
#dataset = dataset.remove_columns('__index_level_0__')

# Map the functions to the dataset
dataset_train = dataset_train.map(set_labels)
dataset_valid = dataset_valid.map(set_labels)

print(dataset_train, dataset_valid)

  0%|          | 0/480 [00:00<?, ?ex/s]

  0%|          | 0/96 [00:00<?, ?ex/s]

Dataset({
    features: ['rewire_id', 'text', 'label', 'label_category', 'label_vector', '__index_level_0__', 'labels'],
    num_rows: 480
}) Dataset({
    features: ['rewire_id', 'text', 'label', 'label_category', 'label_vector', '__index_level_0__', 'labels'],
    num_rows: 96
})


In [17]:
# Reseteamos el formato para que no haya fallos
dataset_train.reset_format() 
dataset_valid.reset_format() 

# Proceso de clasificación

## Tokenización

In [18]:
columns_train = dataset_train.column_names  # Coge todas las columnas
columns_valid = dataset_valid.column_names  # Coge todas las columnas
columns_train.remove("labels") # Elimina la columna "labels"
columns_valid.remove("labels") # Elimina la columna "labels"


## Definición de la métricas

In [19]:
# Función para realizar distintas métricas en ejecución

def compute_metrics(eval_pred):
  """ 
  Compute metrics for Trainer
  """
   
  ############## 
  ## preds son logits, que son tuplas de la forma [valor1, valor2]
  ## Por ejemplo [-1.5606991,  1.6122842] significa que ha predicho eso para un documento
  ## Eso es lo que pasa a la última capa del transformer (softmax si es binario)
  ## Por eso cogemos el índice del valor máximo de la tupla, para decir que esa es la clase que predice
  ##############

  labels = eval_pred.label_ids
  preds = eval_pred.predictions.argmax(-1)

  #preds = np.argmax(preds, axis=-1)
  precision, recall, f1, _ = sk.metrics.precision_recall_fscore_support(labels, preds, average="macro")
  acc = sk.metrics.accuracy_score(labels, preds)
  AUC = roc_auc_score(labels, preds)
  return { 'accuracy': acc, 'f1-score': f1, 'precision': precision, 'recall': recall, 'AUC': AUC }
  #return { 'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [20]:
dataset_valid[:]

{'rewire_id': ['sexism2022_english-10606',
  'sexism2022_english-6991',
  'sexism2022_english-18106',
  'sexism2022_english-18632',
  'sexism2022_english-7027',
  'sexism2022_english-2537',
  'sexism2022_english-3136',
  'sexism2022_english-4548',
  'sexism2022_english-13473',
  'sexism2022_english-2873',
  'sexism2022_english-13237',
  'sexism2022_english-7457',
  'sexism2022_english-16236',
  'sexism2022_english-18627',
  'sexism2022_english-1460',
  'sexism2022_english-7333',
  'sexism2022_english-1322',
  'sexism2022_english-15222',
  'sexism2022_english-456',
  'sexism2022_english-13526',
  'sexism2022_english-10896',
  'sexism2022_english-19608',
  'sexism2022_english-11098',
  'sexism2022_english-14798',
  'sexism2022_english-2238',
  'sexism2022_english-19690',
  'sexism2022_english-2367',
  'sexism2022_english-980',
  'sexism2022_english-15098',
  'sexism2022_english-6457',
  'sexism2022_english-729',
  'sexism2022_english-12055',
  'sexism2022_english-14716',
  'sexism2022_en

## WandDB


In [ ]:
# para usar wandb
# comprobar si los modelos están bien
!pip install wandb
#WANDB_PROJECT="EDO_trainer"
import wandb
wandb.login()
n_labels = 2
epochs = 5
sweep_config = {
    'method': 'random', #grid, random
   # 'metric': {
   #   'name': 'eval/f1',
   #   'goal': 'maximize'   
   # },
    'parameters': {
      #  'epochs':{
      #      'values':[2, 3, 4]
      #  },
        'weight_decay':{'values':[0.1,0.01,0.001]},
        'model': {'values':[#'bert-base-uncased'
                            #'microsoft/deberta-v3-base',
                            #'finiteautomata/bertweet-base-sentiment-analysis',
                            #'roberta-base',
                            #'pysentimiento/robertuito-base',
                            #'nghuyong/ernie-2.0-base-en'
                            #'cardiffnlp/twitter-roberta-base-sentiment'
                            'xlm-roberta-large'
                            ]},
        'max_len' :{'values': [32, 64] },#[32,64]},#[128,64,512]},
        'learning_rate': {
            'values': [ 5e-5, 3e-5, 2e-5]
        },
        'batch_size': {
            'values': [32, 16]
        }
        
    }
}
sweep_id = wandb.sweep(sweep_config,project = "EDO_ENEROFULL")
# Se definen los parámetros del Trainer()


   
def train():
    wandb.init()
    print("******************** nuevo barrido *********************************")
    print("********************************************************************")
    print("config ",wandb.config)
    print("*******************************************************************")
    
   
    tokenizerWandb = AutoTokenizer.from_pretrained(wandb.config.model)
    def tokenize_dataWandb(examples):
               return tokenizerWandb(examples["text"], truncation=True, max_length=wandb.config.max_len, padding=True)
    encoded_dataset_train = dataset_train.map(tokenize_dataWandb, batched=True, remove_columns=columns_train)
    encoded_dataset_valid = dataset_valid.map(tokenize_dataWandb, batched=True, remove_columns=columns_valid)
    num_train_samples = int(len(encoded_dataset_train))
    num_evaluation= int(len(encoded_dataset_valid))
    logging_steps = len(encoded_dataset_train) // (2 * wandb.config.batch_size * epochs)
    optim=["adamw_hf", "adamw_torch", "adamw_apex_fused","adafactor","adamw_torch_xla"]
    print("********************** loggin_steps", logging_steps) 
    print("********************** len", len(encoded_dataset_train)) 
    training_args = TrainingArguments(
      output_dir = 'results',
      num_train_epochs = epochs,
      learning_rate = wandb.config.learning_rate,
      per_device_train_batch_size = wandb.config.batch_size,
      per_device_eval_batch_size = wandb.config.batch_size,
      load_best_model_at_end = True,
      metric_for_best_model = 'f1-score',
      #metric_for_best_model = 'eval_loss',
      weight_decay = wandb.config.weight_decay,
      evaluation_strategy = 'epoch',
      save_strategy = 'epoch',
      logging_steps = logging_steps,
      save_total_limit = 3,
      optim = optim[1],
      push_to_hub=False,
      report_to = "wandb",
      run_name = "prueba"
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-amazon_reviews_multi"
    )
    print("config ",wandb.config)
    def model_init():
        return AutoModelForSequenceClassification.from_pretrained(wandb.config.model, 
                                                           num_labels=2,
                                                           output_attentions = False, # Whether the model returns attentions weights.
                                                           output_hidden_states = False,
                                                           return_dict=True 
                                                           )
    trainer = Trainer(
      model_init=model_init,
      #model = AutoModelSequenceClassification.from_pretrained(model,num_labels=n_labels),
      args = training_args,
      compute_metrics = compute_metrics,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
      train_dataset = encoded_dataset_train,
      #eval_dataset=dataset['valid'],
      eval_dataset = encoded_dataset_valid,
      tokenizer = tokenizerWandb
    )
    trainer.train()

    wandb.finish()
    


   

wandb.agent(sweep_id, function = train, count = 20) 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0050fbbe75b4b0c5519fa189b78834df485c53e9a3aeae2954826f9ab28c263a
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: ex9bwnos
Sweep URL: https://wandb.ai/albertort97/EDO_ENEROFULL/sweeps/ex9bwnos


...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices


********************** loggin_steps 3
********************** len 480
config  {'batch_size': 16, 'learning_rate': 5e-05, 'max_len': 32, 'model': 'xlm-roberta-large', 'weight_decay': 0.001}


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large/snapshots/b2a6150f8be56457baf80c74342cc424080260f0/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large/snapshots